In [117]:
import numpy
import scipy.special
import csv

numpy.random.seed(0)
INPUT_LAYER_SIZE = 785 # 28x28 pixel image = 785 pixels
OUTPUT_LAYER_SIZE = 9 #digits 0-9
HIDDEN_LAYER_SIZE = 16 #arbitrary

In [118]:
train_file = open("mnist_test.csv", mode="r")
train_data = csv.reader(train_file)

In [119]:
class Network:
    def __init__(self, num_inputs, num_neurons):
        self.activation_ReLU = lambda x: numpy.maximum(0, x)
        self.process = lambda inputs: self.activation_ReLU(numpy.dot(inputs,self.weights) + self.biases)
        self.weights = 0.10 * numpy.random.randn(num_inputs, num_neurons)
        self.biases = numpy.zeros((1, num_neurons))

In [120]:
def SoftMax(inputs_layer):
    max_num = numpy.max(inputs_layer, axis=1, keepdims=True)
    exponented = numpy.exp(inputs_layer - max_num)
    normalized = exponented / numpy.sum(exponented, axis=1, keepdims=True)
    return normalized


In [121]:
first_layer = Network(INPUT_LAYER_SIZE, HIDDEN_LAYER_SIZE)
second_layer = Network(HIDDEN_LAYER_SIZE, HIDDEN_LAYER_SIZE)
output_layer = Network(HIDDEN_LAYER_SIZE, OUTPUT_LAYER_SIZE)

In [130]:
output_data = output_layer.process(
    second_layer.process(
        first_layer.process(
            numpy.array(train_data.__next__(), float)
            )
        )
    )
final_result = SoftMax(output_data)
print(final_result)

[[1.02374293e-20 1.87684377e-26 3.76707049e-09 2.06015612e-24
  4.81499866e-21 1.87684377e-26 1.47913924e-12 1.42463024e-11
  9.99999996e-01]]
